d #Demand Forecasting

The objective of this notebook is to illustrate how we might generate a large number of fine-grained forecasts at the store-item level in an efficient manner leveraging the distributed computational power of Databricks.  For this exercise, we will make use of an increasingly popular library for demand forecasting, [FBProphet](https://facebook.github.io/prophet/), which we will load into the notebook session associated with a cluster running Databricks 6.0 or higher:

In [2]:
# load fbprophet library
dbutils.library.installPyPI('FBProphet', version='0.5') # find latest version of fbprophet here: https://pypi.org/project/fbprophet/
dbutils.library.installPyPI('holidays','0.9.12') # this line is in response to this issue with fbprophet 0.5: https://github.com/facebook/prophet/issues/1293

dbutils.library.restartPython()

PyPI package FBProphet has been installed already. The previously-installed package is `FBProphet==0.5`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.
PyPI package holidays has been installed already. The previously-installed package is `holidays==0.9.12`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.

## Examine the Data

For our training dataset, we will make use of 5-years of store-item unit sales data for 50 items across 10 different stores.  This data set is publicly available as part of a past Kaggle competition and can be downloaded [here](https://www.kaggle.com/c/demand-forecasting-kernels-only/data). 

Once downloaded, we can uzip the *train.csv.zip* file and upload the decompressed CSV to */FileStore/tables/demand_forecast/train/* using the file import steps documented [here](https://docs.databricks.com/data/tables.html#create-table-ui). Please note when performing the file import, you don't need to select the *Create Table with UI* or the *Create Table in Notebook* options to complete the import process.

With the dataset accessible within Databricks, we can now explore it in preparation for modeling:

In [4]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/2015.csv,2015.csv,1806322
dbfs:/FileStore/tables/LAcasesus.csv,LAcasesus.csv,27018
dbfs:/FileStore/tables/NewyorkUSCases.csv,NewyorkUSCases.csv,24847
dbfs:/FileStore/tables/airlines.csv,airlines.csv,779982
dbfs:/FileStore/tables/coronadataset.csv,coronadataset.csv,36802504
dbfs:/FileStore/tables/coronadatasetus.csv,coronadatasetus.csv,37492352
dbfs:/FileStore/tables/coronadatasetus2.csv,coronadatasetus2.csv,26265
dbfs:/FileStore/tables/covidts.csv,covidts.csv,1169107
dbfs:/FileStore/tables/customers.csv,customers.csv,735513
dbfs:/FileStore/tables/flights.csv,flights.csv,72088113


In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

import re
from pyspark.sql.types import *

# structure of the training data set
train_schema = StructType([
  StructField('UID', IntegerType()),
  StructField('iso2', StringType()),
  StructField('iso3', StringType()),
  StructField('code3', IntegerType()),
  StructField('FIPS', IntegerType()),
  StructField('Admin2', StringType()),
  StructField('Lat', DoubleType()),
  StructField('CombinedKey', StringType()),
  StructField('Date', StringType()),
  StructField('Case', IntegerType()),
  StructField('Long', DoubleType()),
  StructField('Country', StringType()),
  StructField('Province', StringType())
  ])

# read the training file into a dataframe
#  orignal location:   '/FileStore/tables/demand_forecast/train/train.csv', 
train = spark.read.csv(
  '/FileStore/tables/LAcasesus.csv', 
  header=True, 
  schema=train_schema
  )

# make the dataframe queriable as a temporary view
train.createOrReplaceTempView('train')

In [6]:
train.show()
 


+--------+----+----+-----+----+-----------+-----------+--------------------+----------+----+------------+-------+----------+
 UID|iso2|iso3|code3|FIPS| Admin2| Lat| CombinedKey| Date|Case| Long|Country| Province|
+--------+----+----+-----+----+-----------+-----------+--------------------+----------+----+------------+-------+----------+
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-22| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-23| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-24| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-25| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-26| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-27| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-28| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-29| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-30| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-01-31| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-01| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-02| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-03| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-04| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-05| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-06| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-07| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-08| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-09| 1|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...|2020-02-10| 1|-118.2282411| US|California|
+--------+----+----+-----+----+-----------+-----------+--------------------+----------+----+------------+-------+----------+
only showing top 20 rows

When performing demand forecasting, we are often interested in general trends and seasonality.  Let's start our exploration by examing the annual trend in unit sales:

In [8]:
%sql

SELECT
  year(Date) as year, 
  sum(Case) as case
FROM train
GROUP BY year(Date)
ORDER BY year;

year,case
2020,673613


It's very clear from the data that there is a generally upward trend in total unit sales across the stores. If we had better knowledge of the markets served by these stores, we might wish to identify whether there is a maximum growth capacity we'd expect to approach over the life of our forecast.  But without that knowledge and by just quickly eyeballing this dataset, it feels safe to assume that if our goal is to make a forecast a few days, months or even a year out, we might expect continued linear growth over that time span.

Now let's examine seasonality.  If we aggregate the data around the individual months in each year, a distinct yearly seasonal pattern is observed which seems to grow in scale with overall growth in sales:

In [10]:
%sql

SELECT 
  TRUNC(Date, 'MM') as month,
  SUM(Case) as Case
FROM train
GROUP BY TRUNC(date, 'MM')
ORDER BY month;

month,Case
2020-01-01,6
2020-02-01,29
2020-03-01,18452
2020-04-01,391150
2020-05-01,263976


Aggregating the data at a weekday level, a pronounced weekly seasonal pattern is observed with a peak on Sunday (weekday 0), a hard drop on Monday (weekday 1) and then a steady pickup over the week heading back to the Sunday high.  This pattern seems to be pretty stable across the five years of observations:

In [12]:
%sql

SELECT
  YEAR(Date) as year,
  CAST(DATE_FORMAT(Date, 'u') as Integer) % 7 as weekday,
  --CONCAT(DATE_FORMAT(date, 'u'), '-', DATE_FORMAT(date, 'EEEE')) as weekday,
  AVG(Case) as Case
FROM (
  SELECT 
    Date,
    SUM(Case) as Case
  FROM train
  GROUP BY Date
 ) x
GROUP BY year, CAST(DATE_FORMAT(Date, 'u') as Integer) --, CONCAT(DATE_FORMAT(date, 'u'), '-', DATE_FORMAT(date, 'EEEE'))
ORDER BY year, weekday;

year,weekday,Case
2020,0,5390.466666666666
2020,1,5717.533333333334
2020,2,6097.4
2020,3,6067.1875
2020,4,6358.9375
2020,5,6674.875
2020,6,6869.75


Now that we are oriented to the basic patterns within our data, let's explore how we might build a forecast.

###Build a Forecast

Before attempting to generate forecasts for individual combinations of stores and items, it might be helpful to build a single forecast for no other reason than to orient ourselves to the use of FBProphet.

Our first step is to assemble the historical dataset on which we will train the model:

In [15]:
# query to aggregate data to date (ds) level
sql_statement = '''
  SELECT
    CAST(Date as date) as ds,
    Case as y
  FROM train
  WHERE Country='US' AND Admin2='Los Angeles'
  ORDER BY ds
  '''

# assemble dataset in Pandas dataframe
history_pd = spark.sql(sql_statement).toPandas()

# drop any missing records
history_pd = history_pd.dropna()

Now, we will import the fbprophet library, but because it can be a bit verbose when in use, we will need to fine-tune the logging settings in our environment:

In [17]:
from fbprophet import Prophet
import logging

# disable informational messages from fbprophet
logging.getLogger('py4j').setLevel(logging.ERROR)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.

Based on our review of the data, it looks like we should set our overall growth pattern to linear and enable the evaluation of weekly and yearly seasonal patterns. We might also wish to set our seasonality mode to multiplicative as the seasonal pattern seems to grow with overall growth in sales:

In [19]:
# set model parameters
model = Prophet(
  interval_width=0.95,
  growth='linear',
  daily_seasonality=False,
  weekly_seasonality=True,
  yearly_seasonality=True,
  seasonality_mode='multiplicative'
  )

# fit the model to historical data
model.fit(history_pd)

Out[5]: <fbprophet.forecaster.Prophet at 0x7fda2354b080>

Now that we have a trained model, let's use it to build a 90-day forecast:

In [21]:
# define a dataset including both historical dates & 90-days beyond the last available date
future_pd = model.make_future_dataframe(
  periods=90, 
  freq='d', 
  include_history=True
  )

# predict over the dataset
forecast_pd = model.predict(future_pd)

display(forecast_pd)

ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
2020-01-22T00:00:00.000+0000,-7964.461075694761,-639.211474137079,365.06014318878755,-7964.461075694761,-7964.461075694761,-0.9808838339769206,-0.9808838339769206,-0.9808838339769206,0.0088047782574063,0.0088047782574063,0.0088047782574063,-0.9896886122343269,-0.9896886122343269,-0.9896886122343269,0.0,0.0,0.0,-152.24996020733474
2020-01-23T00:00:00.000+0000,-7665.545841453902,-500.1937562028124,500.22669027926503,-7665.545841453902,-7665.545841453902,-0.9977148804030792,-0.9977148804030792,-0.9977148804030792,0.007314853459861725,0.007314853459861725,0.007314853459861725,-1.005029733862941,-1.005029733862941,-1.005029733862941,0.0,0.0,0.0,-17.516689023401
2020-01-24T00:00:00.000+0000,-7366.63060721304,-419.0644768906492,524.6849933980769,-7366.63060721304,-7366.63060721304,-1.0057497624539662,-1.0057497624539662,-1.0057497624539662,0.007571775534211263,0.007571775534211263,0.007571775534211263,-1.0133215379881775,-1.0133215379881775,-1.0133215379881775,0.0,0.0,0.0,42.35637607759169
2020-01-25T00:00:00.000+0000,-7067.715372972179,-295.539358369863,628.8677412311988,-7067.715372972179,-7067.715372972179,-1.020870045872263,-1.020870045872263,-1.020870045872263,-0.004918253750966221,-0.004918253750966221,-0.004918253750966221,-1.0159517921212968,-1.0159517921212968,-1.0159517921212968,0.0,0.0,0.0,147.50354404602808
2020-01-26T00:00:00.000+0000,-6768.800138775587,-305.442155572624,666.8886954267061,-6768.800138775587,-6768.800138775587,-1.025497909190693,-1.025497909190693,-1.025497909190693,-0.011204092935845998,-0.011204092935845998,-0.011204092935845998,-1.014293816254847,-1.014293816254847,-1.014293816254847,0.0,0.0,0.0,172.5902512684505
2020-01-27T00:00:00.000+0000,-6469.884904578994,-354.4911111128165,585.0033575941228,-6469.884904578994,-6469.884904578994,-1.0183736311892155,-1.0183736311892155,-1.0183736311892155,-0.008707754178094952,-0.008707754178094952,-0.008707754178094952,-1.0096658770111206,-1.0096658770111206,-1.0096658770111206,0.0,0.0,0.0,118.87527907340737
2020-01-28T00:00:00.000+0000,-6170.9696703824,-456.46125879781334,510.3889440054493,-6170.9696703824,-6170.9696703824,-1.0021564419248923,-1.0021564419248923,-1.0021564419248923,0.0011386936134446157,0.0011386936134446157,0.0011386936134446157,-1.0032951355383368,-1.0032951355383368,-1.0032951355383368,0.0,0.0,0.0,13.30733771445132
2020-01-29T00:00:00.000+0000,-5872.054436185807,-538.5825532417073,424.78381765138835,-5872.054436185807,-5872.054436185807,-0.9874822187542329,-0.9874822187542329,-0.9874822187542329,0.008804778257418476,0.008804778257418476,0.008804778257418476,-0.9962869970116515,-0.9962869970116515,-0.9962869970116515,0.0,0.0,0.0,-73.50509289541007
2020-01-30T00:00:00.000+0000,-5573.139202060005,-613.7602973461619,418.25561902043097,-5573.139202060005,-5573.139202060005,-0.982285650528678,-0.982285650528678,-0.982285650528678,0.0073148534598643355,0.0073148534598643355,0.0073148534598643355,-0.9896005039885423,-0.9896005039885423,-0.9896005039885423,0.0,0.0,0.0,-98.72453547761556
2020-01-31T00:00:00.000+0000,-5274.223967934202,-626.1185134147001,379.9602469997419,-5274.223967934202,-5274.223967934202,-0.9764584230327724,-0.9764584230327724,-0.9764584230327724,0.007571775534230173,0.007571775534230173,0.007571775534230173,-0.9840301985670026,-0.9840301985670026,-0.9840301985670026,0.0,0.0,0.0,-124.16354948351949


How did our model perform? Here we can see the general and seasonal trends in our model presented as graphs:

In [23]:
trends_fig = model.plot_components(forecast_pd)
display(trends_fig)

And here, we can see how our actual and predicted data line up as well as a forecast for the future, though we will limit our graph to the last year of historical data just to keep it readable:

In [25]:
predict_fig = model.plot( forecast_pd, xlabel='Date', ylabel='Case')

# adjust figure to display dates from last year + the 90 day forecast
xlim = predict_fig.axes[0].get_xlim()
new_xlim = ( xlim[1]-(245.0), xlim[1]-45.0)
predict_fig.axes[0].set_xlim(new_xlim)

display(predict_fig)

**NOTE** This visualization is a bit busy. Bartosz Mikulski provides [an excellent breakdown](https://www.mikulskibartosz.name/prophet-plot-explained/) of it that is well worth checking out.  In a nutshell, the black dots represent our actuals with the darker blue line representing our predictions and the lighter blue band representing our (95%) uncertainty interval.

Visual inspection is useful, but a better way to evaulate the forecast is to calculate Mean Absolute Error, Mean Squared Error and Root Mean Squared Error values for the predicted relative to the actual values in our set:

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from datetime import date

# get historical actuals & predictions for comparison
actuals_pd = history_pd[ history_pd['ds'] < date(2020, 5, 6) ]['y']
predicted_pd = forecast_pd[ forecast_pd['ds'] < date(2020, 5, 6) ]['yhat']

# calculate evaluation metrics
mae = mean_absolute_error(actuals_pd, predicted_pd)
mse = mean_squared_error(actuals_pd, predicted_pd)
rmse = sqrt(mse)

# print metrics to the screen
print( '\n'.join(['MAE: {0}', 'MSE: {1}', 'RMSE: {2}']).format(mae, mse, rmse) )

/local_disk0/tmp/1589214853828-0/PythonShell.py:7: FutureWarning: Comparing Series of datetimes with 'datetime.date'. Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
 import pickle
MAE: 181.63966080090984
MSE: 64984.73516898168
RMSE: 254.9210371251884

FBProphet provides [additional means](https://facebook.github.io/prophet/docs/diagnostics.html) for evaluating how your forecasts hold up over time. You're strongly encouraged to consider using these and those additional techniques when building your forecast models but we'll skip this here to focus on the scaling challenge.

###Scaling Model Training & Forecasting

With the mechanics under our belt, let's now tackle our original goal of building numerous, fine-grain models & forecasts for individual store and item combinations.  We will start by assembling sales data at the store-item-date level of granularity:

**NOTE**: The data in this data set should already be aggregated at this level of granularity but we are explicitly aggregating to ensure we have the expected data structure.

In [31]:
sql_statement = '''
  SELECT
    Province,
    Admin2,
    CAST(Date as Date) as ds,
    SUM(Case) as y
  FROM train
  GROUP BY Province, Admin2, ds
  ORDER BY Province, Admin2, ds
  '''

store_item_history = (
  spark
    .sql( sql_statement )
    .repartition(sc.defaultParallelism, ['Province', 'Admin2'])
  ).cache()

With our data aggregated at the store-item-date level, we need to consider how we will pass our data to FBProphet. If our goal is to build a model for each store and item combination, we will need to pass in a store-item subset from the dataset we just assembled, train a model on that subset, and receive a store-item forecast back. We'd expect that forecast to be returned as a dataset with a structure like this where we retain the store and item identifiers for which the forecast was assembled and we limit the output to just the relevant subset of fields generated by the Prophet model:

In [33]:
from pyspark.sql.types import *

result_schema =StructType([
  StructField('ds',DateType()),
  StructField('Province',StringType()),
  StructField('Admin2',StringType()),
  StructField('y',FloatType()),
  StructField('yhat',FloatType()),
  StructField('yhat_upper',FloatType()),
  StructField('yhat_lower',FloatType())
  ])

To train the model and generate a forecast we will leverage a Pandas user-defined function (UDF).  We will define this function to receive a subset of data organized around a store and item combination.  It will return a forecast in the format identified in the previous cell:

In [35]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf( result_schema, PandasUDFType.GROUPED_MAP )
def forecast_store_item( history_pd ):
  
  # TRAIN MODEL AS BEFORE
  # --------------------------------------
  # remove missing values (more likely at day-store-item level)
  history_pd = history_pd.dropna()
  
  # configure the model
  model = Prophet(
    interval_width=0.95,
    growth='linear',
    daily_seasonality=False,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='multiplicative'
    )
  
  # train the model
  model.fit( history_pd )
  # --------------------------------------
  
  # BUILD FORECAST AS BEFORE
  # --------------------------------------
  # make predictions
  future_pd = model.make_future_dataframe(
    periods=90, 
    freq='d', 
    include_history=True
    )
  forecast_pd = model.predict( future_pd )  
  # --------------------------------------
  
  # ASSEMBLE EXPECTED RESULT SET
  # --------------------------------------
  # get relevant fields from forecast
  f_pd = forecast_pd[ ['ds','yhat', 'yhat_upper', 'yhat_lower'] ].set_index('ds')
  
  # get relevant fields from history
  h_pd = history_pd[['ds','Province','Admin2','y']].set_index('ds')
  
  # join history and forecast
  results_pd = f_pd.join( h_pd, how='left' )
  results_pd.reset_index(level=0, inplace=True)
  
  # get store & item from incoming data set
  results_pd['Province'] = history_pd['Province'].iloc[0]
  results_pd['Admin2'] = history_pd['Admin2'].iloc[0]
  # --------------------------------------
  
  # return expected dataset
  return results_pd[ ['ds', 'Province', 'Admin2', 'y', 'yhat', 'yhat_upper', 'yhat_lower'] ]  

There's a lot taking place within our UDF, but if you compare the first two blocks of code within which the model is being trained and a forecast is being built to the cells in the previous portion of this notebook, you'll see the code is pretty much the same as before. It's only in the assembly of the required result set that truly new code is being introduced and it consists of fairly standard Pandas dataframe manipulations.

Now let's call our UDF to build our forecasts.  We do this by grouping our historical dataset around store and item.  We then apply our UDF to each group and tack on today's date as our *training_date* for data management purposes:

In [38]:
from pyspark.sql.functions import current_date

results = (
  store_item_history
    .groupBy('Province', 'Admin2')
    .apply(forecast_store_item)
    .withColumn('training_date', current_date() )
    )

results.createOrReplaceTempView('new_forecasts3')

In [39]:
%sql
Select * from new_forecasts3;

ds,Province,Admin2,y,yhat,yhat_upper,yhat_lower,training_date
2020-01-22,California,Los Angeles,0.0,-152.24995,337.3573,-673.27325,2020-05-11
2020-01-23,California,Los Angeles,0.0,-17.51669,503.02066,-524.33826,2020-05-11
2020-01-24,California,Los Angeles,0.0,42.356377,551.8403,-453.9266,2020-05-11
2020-01-25,California,Los Angeles,0.0,147.50354,648.79565,-345.25186,2020-05-11
2020-01-26,California,Los Angeles,1.0,172.59026,663.48315,-337.14136,2020-05-11
2020-01-27,California,Los Angeles,1.0,118.87528,597.48834,-370.49152,2020-05-11
2020-01-28,California,Los Angeles,1.0,13.307338,513.8488,-503.2879,2020-05-11
2020-01-29,California,Los Angeles,1.0,-73.5051,398.22827,-579.2945,2020-05-11
2020-01-30,California,Los Angeles,1.0,-98.72453,393.26114,-568.3816,2020-05-11
2020-01-31,California,Los Angeles,1.0,-124.16355,378.19452,-618.6128,2020-05-11


We we are likely wanting to report on our forecasts, so let's save them to a queriable table structure:

In [41]:
%sql
-- create forecast table
create table if not exists forecasts3 (
  date date,
  Province string,
  Admin2 string,
  case float,
  case_predicted float,
  case_predicted_upper float,
  case_predicted_lower float,
  training_date date
  )
using delta
partitioned by (training_date);



In [42]:
%sql
select 
  ds as date,
  Province,
  Admin2,
  y as case,
  yhat as case_predicted,
  yhat_upper as case_predicted_upper,
  yhat_lower as case_predicted_lower,
  training_date
from new_forecasts3;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,California,Los Angeles,0.0,-152.24995,335.2836,-676.48267,2020-05-11
2020-01-23,California,Los Angeles,0.0,-17.51669,447.652,-508.2438,2020-05-11
2020-01-24,California,Los Angeles,0.0,42.356377,523.75867,-456.87488,2020-05-11
2020-01-25,California,Los Angeles,0.0,147.50354,650.5091,-345.71002,2020-05-11
2020-01-26,California,Los Angeles,1.0,172.59026,643.08185,-346.1188,2020-05-11
2020-01-27,California,Los Angeles,1.0,118.87528,610.1044,-409.79156,2020-05-11
2020-01-28,California,Los Angeles,1.0,13.307338,522.1255,-522.5149,2020-05-11
2020-01-29,California,Los Angeles,1.0,-73.5051,403.27344,-541.4426,2020-05-11
2020-01-30,California,Los Angeles,1.0,-98.72453,401.49384,-575.0526,2020-05-11
2020-01-31,California,Los Angeles,1.0,-124.16355,357.06793,-637.92017,2020-05-11


In [43]:
 %sql
-- load data to it
insert into forecasts3
select 
  ds as date,
  Province,
  Admin2,
 y as case,
  yhat as case_predicted,
  yhat_upper as case_predicted_upper,
  yhat_lower as case_predicted_lower,
  training_date
from new_forecasts3;

In [44]:
%sql
Select * from forecasts3;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,California,Los Angeles,0.0,-152.24995,343.31967,-641.3819,2020-05-11
2020-01-23,California,Los Angeles,0.0,-17.51669,478.38116,-513.77216,2020-05-11
2020-01-24,California,Los Angeles,0.0,42.356377,514.23425,-443.79965,2020-05-11
2020-01-25,California,Los Angeles,0.0,147.50354,662.00934,-321.50568,2020-05-11
2020-01-26,California,Los Angeles,1.0,172.59026,677.5602,-335.92245,2020-05-11
2020-01-27,California,Los Angeles,1.0,118.87528,614.6287,-397.68387,2020-05-11
2020-01-28,California,Los Angeles,1.0,13.307338,540.5777,-492.36584,2020-05-11
2020-01-29,California,Los Angeles,1.0,-73.5051,402.80188,-558.2108,2020-05-11
2020-01-30,California,Los Angeles,1.0,-98.72453,414.70078,-662.8998,2020-05-11
2020-01-31,California,Los Angeles,1.0,-124.16355,414.35605,-650.96594,2020-05-11


But how good (or bad) is each forecast?  Using the UDF technique, we can generate evaluation metrics for each store-item forecast as follows:

In [46]:
import pandas as pd

# schema of expected result set
eval_schema =StructType([
  StructField('training_date', DateType()),
  StructField('Province', StringType()),
  StructField('Admin2', StringType()),
  StructField('mae', FloatType()),
  StructField('mse', FloatType()),
  StructField('rmse', FloatType())
  ])

# define udf to calculate metrics
@pandas_udf( eval_schema, PandasUDFType.GROUPED_MAP )
def evaluate_forecast( evaluation_pd ):
  
  # get store & item in incoming data set
  training_date = evaluation_pd['training_date'].iloc[0]
  Province = evaluation_pd['Province'].iloc[0]
  Admin2 = evaluation_pd['Admin2'].iloc[0]
  
  # calulate evaluation metrics
  mae = mean_absolute_error( evaluation_pd['y'], evaluation_pd['yhat'] )
  mse = mean_squared_error( evaluation_pd['y'], evaluation_pd['yhat'] )
  rmse = sqrt( mse )
  
  # assemble result set
  results = {'training_date':[training_date], 'Province':[Province], 'Admin2':[Admin2], 'mae':[mae], 'mse':[mse], 'rmse':[rmse]}
  return pd.DataFrame.from_dict( results )

# calculate metrics
results = (
  spark
    .table('new_forecasts3')
    .filter('ds < \'2020-05-06\'') # limit evaluation to periods where we have historical data
    .select('training_date', 'Province', 'Admin2', 'y', 'yhat')
    .groupBy('training_date', 'Province', 'Admin2')
    .apply(evaluate_forecast)
    )
results.createOrReplaceTempView('new_forecast_evals3')



Once again, we will likely want to report the metrics for each forecast, so we persist these to a queriable table:

In [48]:
%sql

create table if not exists forecast_evals3 (
  Province string,
  Admin2 string,
  mae float,
  mse float,
  rmse float,
  training_date date
  )
using delta
partitioned by (training_date);

insert into forecast_evals3
select
  Province,
  Admin2,
  mae,
  mse,
  rmse,
  training_date
from new_forecast_evals3;

In [49]:
%sql
Select * from forecasts3;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,California,Los Angeles,0.0,-152.24995,343.31967,-641.3819,2020-05-11
2020-01-23,California,Los Angeles,0.0,-17.51669,478.38116,-513.77216,2020-05-11
2020-01-24,California,Los Angeles,0.0,42.356377,514.23425,-443.79965,2020-05-11
2020-01-25,California,Los Angeles,0.0,147.50354,662.00934,-321.50568,2020-05-11
2020-01-26,California,Los Angeles,1.0,172.59026,677.5602,-335.92245,2020-05-11
2020-01-27,California,Los Angeles,1.0,118.87528,614.6287,-397.68387,2020-05-11
2020-01-28,California,Los Angeles,1.0,13.307338,540.5777,-492.36584,2020-05-11
2020-01-29,California,Los Angeles,1.0,-73.5051,402.80188,-558.2108,2020-05-11
2020-01-30,California,Los Angeles,1.0,-98.72453,414.70078,-662.8998,2020-05-11
2020-01-31,California,Los Angeles,1.0,-124.16355,414.35605,-650.96594,2020-05-11


We now have constructed a forecast for each store-item combination and generated basic evaluation metrics for each.  To see this forecast data, we can issue a simple query (limited here to product 1 across stores 1 through 10):

In [51]:
%sql

SELECT
  Province,
  date,
  Admin2,
  case_predicted,
  case_predicted_upper,
  case_predicted_lower
FROM forecasts3 a
WHERE Admin2 = 'Los Angeles' AND
      --store IN (1, 2, 3, 4, 5) AND
      date >= '2020-05-01' AND
      training_date=current_date()
ORDER BY Province

Province,date,Admin2,case_predicted,case_predicted_upper,case_predicted_lower
California,2020-05-01,Los Angeles,24366.584,24855.773,23827.217
California,2020-05-02,Los Angeles,24961.965,25460.428,24426.09
California,2020-05-03,Los Angeles,25673.756,26169.24,25190.43
California,2020-05-04,Los Angeles,26573.62,27107.04,26067.52
California,2020-05-05,Los Angeles,27644.117,28170.646,27162.438
California,2020-05-06,Los Angeles,28672.46,29138.174,28143.398
California,2020-05-07,Los Angeles,29499.809,29984.936,29011.717
California,2020-05-08,Los Angeles,30409.701,30893.703,29909.494
California,2020-05-09,Los Angeles,31059.715,31539.385,30575.078
California,2020-05-10,Los Angeles,31952.256,32424.951,31441.64


And for each of these, we can retrieve a measure of help us assess the reliability of each forecast:

In [53]:
%sql

SELECT
  Province,
  mae,
  mse,
  rmse
FROM forecast_evals3 a
WHERE Admin2 = 'Los Angeles' AND
      training_date=current_date()
ORDER BY Province

Province,mae,mse,rmse
California,181.63968,64984.754,254.92107
